In [1]:
import pandas as pd
from arise_prototype import capacity_check as cc

In [2]:
start = '2021-05-15 00:00:00'
end = '2021-05-20 00:00:00'

# Kalkulieren der Produktionsdaten

In [3]:

df, df_workload_capacity = cc.run_capacity_check(start, end)
df


TODO: Check if date and machine combination is unique in            Maschinenschichten


,Unnamed: 0,Fefco_Teil,ArtNr_Teil,ID_Druck,Druckflaeche,Bogen_Laenge_Brutto,Bogen_Breite_Brutto,MaschNr,Start,Ende,...,TeilNr,SchrittNr,Summe_Minuten,ID_Maschstatus,Maschstatus,Lieferdatum_Rohmaterial,BE_Erledigt,Production_date,machine,machine_id
78483,78483,0201 G,20783,0,NaN,1247.0,360.0,SL 5,2021-05-05 08:50:16,2021-05-05 08:50:24,...,1,1,0.0,1,Produktion,2021-05-07 00:00:00,1.0,2021-05-14,SL 5,5
79084,79084,0110 ST,24502,1,0.15,1645.0,1045.0,SL 9 ROTA,2021-05-06 13:30:48,2021-05-06 13:42:29,...,1,1,12.0,1,Produktion,2021-05-10 00:00:00,1.0,2021-05-17,SL 9,9
79107,79107,0110 ST,24503,1,0.20,1630.0,1132.0,SL 9 ROTA,2021-05-06 14:51:01,2021-05-06 15:19:15,...,1,1,28.0,1,Produktion,2021-05-10 00:00:00,1.0,2021-05-17,SL 9,9
130347,130347,0201 G man,19137,1,0.10,2822.0,422.0,SL 2 Rota,2021-05-19 06:22:51,2021-05-19 06:27:40,...,1,1,5.0,1,Produktion,2021-05-12 00:00:00,1.0,2021-05-17,SL 2,2
130349,130349,0201 G man,19137,1,0.10,2822.0,422.0,SL 2 Rota,2021-05-19 06:33:05,2021-05-19 06:50:08,...,1,1,17.0,1,Produktion,2021-05-12 00:00:00,1.0,2021-05-17,SL 2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511669,511669,0201 G,31681,1,0.10,1069.0,373.0,SL 5,2021-05-10 17:26:15,2021-05-10 17:40:37,...,1,1,14.0,1,Produktion,2021-05-06 00:00:00,1.0,2021-05-14,SL 5,5
511674,511674,0201 G,19929,0,NaN,1535.0,576.0,SL 5,2021-05-10 17:51:22,2021-05-10 18:36:01,...,1,1,45.0,1,Produktion,2021-05-06 00:00:00,1.0,2021-05-14,SL 5,5
511691,511691,0201 G,23918,1,0.02,1423.0,404.0,SL 5,2021-05-10 20:43:51,2021-05-10 21:49:01,...,1,1,66.0,1,Produktion,2021-05-11 00:00:00,1.0,2021-05-14,SL 5,5
513409,513409,0110 Flachbett,33359,1,0.10,1244.0,485.0,SL 10,2021-05-26 06:16:08,2021-05-26 07:18:55,...,1,1,62.0,1,Produktion,2021-05-11 00:00:00,1.0,2021-05-14,SL 10,10


TODO: Check if date and machine combination is unique in            Maschinenschichten


,MaschNr,Laufzeit_Soll,LTermin,KndNr,Suchname,AKNR,ArtNr_Teil,TeilNr,SchrittNr,Production_date,machine,machine_id
78483,SL 5,9,2021-05-17,11722,MUECHER,118205,20783,1,1,2021-05-14,SL 5,5
79084,SL 9 ROTA,27,2021-05-18,10883,BETTE,117702,24502,1,1,2021-05-17,SL 9,9
79107,SL 9 ROTA,15,2021-05-18,10883,BETTE,117704,24503,1,1,2021-05-17,SL 9,9
130347,SL 2 Rota,780,2021-05-18,13907,CARTO RAAD,117096,19137,1,1,2021-05-17,SL 2,2
130354,SL 2,54,2021-05-18,10495,KAIMANN,118322,10219,1,2,2021-05-17,SL 2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
511662,SL 5,75,2021-05-17,12381,LEFELD,118305,32445,1,1,2021-05-14,SL 5,5
511669,SL 5,10,2021-05-17,12403,KOETTER,117740,31681,1,1,2021-05-14,SL 5,5
511674,SL 5,28,2021-05-17,12376,GUNDLACH,119245,19929,1,1,2021-05-14,SL 5,5
511691,SL 5,40,2021-05-17,13074,KOECKERLING,118468,23918,1,1,2021-05-14,SL 5,5


In [4]:
df_workload_capacity

,machine_id,date,workload,capacity
0,5,2021-05-14,2431,600
1,9,2021-05-14,1089,600
2,2,2021-05-14,959,959
3,4,2021-05-14,1247,600
4,8,2021-05-14,561,600
5,6,2021-05-14,20262,1019
6,10,2021-05-14,142,600
7,5,2021-05-17,28,750
8,9,2021-05-17,79,750
9,2,2021-05-17,12144,750


# Vergleich der Kapazität mit der Produktionszeit

In [5]:
def compare_capacity(df):
    for index, row in df.iterrows():
        if int(row['workload']) > int(row['capacity']):
            print('Kapazität an SL ', row['machine_id'], ' am', row['date'], ' überschritten.')
            print('Kapazität: ', row['capacity'])
            print('Produktionszeit für geplante Aufträge: ', row['workload'])


In [6]:
compare_capacity(df_workload_capacity)

Kapazität an SL  5  am 2021-05-14 00:00:00  überschritten.
Kapazität:  600
Produktionszeit für geplante Aufträge:  2431
Kapazität an SL  9  am 2021-05-14 00:00:00  überschritten.
Kapazität:  600
Produktionszeit für geplante Aufträge:  1089
Kapazität an SL  4  am 2021-05-14 00:00:00  überschritten.
Kapazität:  600
Produktionszeit für geplante Aufträge:  1247
Kapazität an SL  6  am 2021-05-14 00:00:00  überschritten.
Kapazität:  1019
Produktionszeit für geplante Aufträge:  20262
Kapazität an SL  2  am 2021-05-17 00:00:00  überschritten.
Kapazität:  750
Produktionszeit für geplante Aufträge:  12144
Kapazität an SL  6  am 2021-05-18 00:00:00  überschritten.
Kapazität:  1168
Produktionszeit für geplante Aufträge:  1245
